In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf
import datetime as dt
import matplotlib.pyplot as plt
import math
import datetime


Download the stock dataset.

In [ ]:
#the start and end date
start_date = dt.datetime(2013,4,1)
end_date = dt.datetime(2024,5,3)

#loading from yahoo finance
data = yf.download("GOOGL",start_date, end_date)

Print the first 5 rows to get a brief overview of the data. If you can't remember the name of the function, think about what your gf should give you every night.

In [ ]:
# TODO

# Preprocessing steps

1. Only select the column "Close", bc we only care about that data

In [ ]:
# TODO

2. Plot the column with matplotlib to see the graph

In [ ]:
# TODO

3. Write a function to convert a string to a datetime object. The date string looks like that: "2014-04-01"

In [ ]:
def str_to_datetime(s):
  # Split the string in 3 parts with '-' as a delimiter
  split = # TODO
  year, month, day = # TODO
  return datetime.datetime(year=year, month=month, day=day)

# Data preparation

This function creates multiple subsets with a specific length n. This is needed to convert the problem into a supervised learning problem. We have a target date and n-values before the close price of the target date.

In [ ]:
def df_to_windowed_df(dataframe, first_date_str, last_date_str, n=3):
  first_date = str_to_datetime(first_date_str)
  last_date  = str_to_datetime(last_date_str)

  target_date = first_date

  dates = []
  X, Y = [], []

  last_time = False
  while True:
    df_subset = dataframe.loc[:target_date].tail(n+1)

    if len(df_subset) != n+1:
      print(f'Error: Window of size {n} is too large for date {target_date}')
      return

    values = df_subset['Close'].to_numpy()
    x, y = values[:-1], values[-1]

    dates.append(target_date)
    X.append(x)
    Y.append(y)

    next_week = dataframe.loc[target_date:target_date+datetime.timedelta(days=7)]
    next_datetime_str = str(next_week.head(2).tail(1).index.values[0])
    next_date_str = next_datetime_str.split('T')[0]
    year_month_day = next_date_str.split('-')
    year, month, day = year_month_day
    next_date = datetime.datetime(day=int(day), month=int(month), year=int(year))

    if last_time:
      break

    target_date = next_date

    if target_date == last_date:
      last_time = True

  ret_df = pd.DataFrame({})
  ret_df['Target Date'] = dates

  X = np.array(X)
  for i in range(0, n):
    X[:, i]
    ret_df[f'Target-{n-i}'] = X[:, i]

  ret_df['Target'] = Y

  return ret_df

def windowed_df_to_date_X_y(windowed_dataframe):
  df_as_np = windowed_dataframe.to_numpy()

  dates = df_as_np[:, 0]

  middle_matrix = df_as_np[:, 1:-1]
  X = middle_matrix.reshape((len(dates), middle_matrix.shape[1], 1))

  Y = df_as_np[:, -1]

  return dates, X.astype(np.float32), Y.astype(np.float32)

Example: Window size of 5 and the data between 2018-04-01 and 2024-05-02

In [ ]:
WINDOW_SIZE = 5
# Start day: '2018-04-01', last day: '2024-05-02'
windowed_df = df_to_windowed_df(data,
                                "2018-04-01",
                                "2024-05-02",
                                WINDOW_SIZE)
print(windowed_df)

# Extract dates, x and y
dates, X, y = windowed_df_to_date_X_y(windowed_df)

2. Divide the dataset into training (90%), validation (5%) and test(5%) set.

In [ ]:
dates_train, X_train, y_train = #TODO

dates_val, X_val, y_val = #TODO
dates_test, X_test, y_test = #TODO


plt.plot(dates_train, y_train)
plt.plot(dates_val, y_val)
plt.plot(dates_test, y_test)

plt.legend(['Train', 'Validation', 'Test'])

# Train the model



1. Create a sequential model with an input layer that has the size of the window (WINDOW_SIZE), an LSTM layer with 64 neurons, a dense layer with 32 neurons and Relu activation function and a final dense layer with 1 neuron.

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers

MAX_EPOCHS = 30


model = Sequential([layers.Input((WINDOW_SIZE, 1)),
                    layers.LSTM(),
                    layers.Dense(),
                    layers.Dense()])


2. Compile the model with "mse" as loss function, the optimizer is gonna be "Adam" with a learning rate of 0.001 and the metrics are "mean_absolute_error"

In [ ]:
from tensorflow.keras.optimizers import Adam

model.compile(loss='',
              optimizer=,
              metrics=['mean_absolute_error'])


model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=MAX_EPOCHS)

# Predictions

1. Predict the stock price on the validation dataset

In [ ]:
val_predictions = #TODO

plt.plot(dates_val, val_predictions)
plt.plot(dates_val, y_val)
plt.legend(['Validation Predictions', 'Validation Observations'])

2. Predict the stock price on the test dataset

In [ ]:
test_predictions = # TODO

plt.plot(dates_test, test_predictions)
plt.plot(dates_test, y_test)
plt.legend(['Testing Predictions', 'Testing Observations'])

Overview of all predictions

In [ ]:
plt.plot(dates_val, val_predictions)
plt.plot(dates_val, y_val)
plt.plot(dates_test, test_predictions)
plt.plot(dates_test, y_test)
plt.legend(['Validation Predictions',
            'Validation Observations',
            'Testing Predictions',
            'Testing Observations'])